In [1]:
import konlpy as kn
from ekonlpy.tag import Mecab
import ekonlpy as ek
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from glob import glob

In [2]:
df_dic = pd.read_csv('../DATA/NBC_step3(dictionary)_final-jh-190529.csv',encoding='UTF-8', names=['hawki','dovis'])
#딕셔너리 파일을 불러옴
hawki_ls = df_dic['hawki']
#매파 단어들에 대한 리스트 생성
dovis_ls = df_dic['dovis']
#비둘기파 단어들에 대한 리스트 생성

In [3]:
df = pd.read_csv('../DATA/minutes_preprocessed.csv', encoding='utf-8', names=['date','minutes','Economic Situation','Foreign Currency','Financial Markets','Monetary Policy','Participant Views','Government View','Economic Situation count','Foreign Currency count','Financial Markets count','Monetary Policy count','Participant Views count','Government View count','Foreign Currency tokens','Financial Markets tokens','Foreign Currency ngram','Financial Markets ngram'])
#정제된 의사록 데이터 불러옴
df = df.sort_values(by = ['date'], axis=0)
#날짜 순으로 정렬
df = df.drop(['Economic Situation','Monetary Policy','Participant Views','Government View', 'Economic Situation count','Monetary Policy count','Participant Views count','Government View count'], axis=1)
df = df.drop(['minutes','Foreign Currency count','Financial Markets count','Foreign Currency tokens', 'Financial Markets tokens','Foreign Currency ngram','Financial Markets ngram'],axis=1)
#의사록 데이터의 섹션 2,3만 사용
df = df.drop(0)
df

,date,Foreign Currency,Financial Markets
1,20050609,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...
2,20050707,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...
3,20050811,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...
4,20050908,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...
5,20051011,일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...
6,20051110,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",일부 위원은 장단기금리 격차가 경제성장률에 선행하는 모습을 보인다고 하면 미국의...
7,20051208,일부 위원은 금년 들어 달러화가 엔화 나 유로화에 대해 상당히 강세를 나타낸 것과...,일부 위원은 우리나라와 미국의 장기시장금리 를 비교해볼 때 최근 우리나라의 장기시...
8,20060112,일부 위원은 2002년부터 거의 예외 없 이 연초에는 원화의 강세 현상이 나타나 ...,일부 위원은 실물활동 중 민간소비나 GDP 증가율이 M1증가율과 밀접한 관련성이...
9,20060209,일부 위원은 자본유입에 대해서는 관대 한 반면 자본유출에 대해서는 비교적 엄격한 ...,일부 위원은 최근 주가가 크게 변동한 이유 중 하나로 기관투자가들이 단기적 관점...
10,20060309,일부 위원은 과거 사례를 보면 원화환 율은 엔화환율 및 위안화환율과 높은 상관관계...,일부 위원은 지난 해 10월 이후 세 차례에 걸 친 콜금리 목표 인상에도 불구하고...


In [4]:
For_ls = df['Foreign Currency']
Fin_ls = df['Financial Markets']
tmp_ls = For_ls + Fin_ls
df['For+Fin'] = tmp_ls
df = df.dropna()
df = df.drop(columns=['Foreign Currency', 'Financial Markets'])
tmp_ls[1]

'일부 위원은 우리나라 경제가 일본경제  에 비해 더 나아질 특별한 요인이 없음에도 불구하고 최근 원/엔 환율이 지속적으  로 하락하는 이유를 물었으며, 이에 대해 관련부서에서는 위안화 절상시 동아시아  권 통화의 절상폭이 클 것이라는 기대로 원화나 대만달러가 엔/달러 환율의 상승  을 추수하지 못하고 국내 외환수급 면에서 공급우위 상태가 지속되고 있는 데다   국제 투자은행들이 엔/달러 환율 전망을 상향조정하고 있음에도 불구하고 원/달러   환율에 대한 조정은 아직 이루어지지 않고 있어 기업 등 시장참가자들의 기대도   원화 절상 쪽으로 편향되어 있는 데 기인한다고 설명하였음.@@@- 6 -     동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamental)을 반영하  여 움직인다는 점을 감안할 때 현재의 원/엔 환율 하락은 일시적인 현상이며 장기  적으로 반전될 가능성이 있는 것으로 보인다는 견해를 밝혔으며, 이에 대해 관련  부서에서는 동 견해에 동의하면서 우리나라의 경우 시장규모가 작아 일시적인 외  부 충격에 의한 영향을 쉽게 받는다는 점을 설명하는 한편, 향후 우리나라와 미국  간 내외금리 역전이 나타나고 작년과 같은 대규모 외환 유입은 일어나지 않을 것  임을 감안할 때 원/엔 환율이 추가 하락할 여지는 점차 축소될 것으로 생각된다고   답변하였음.@@@다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운  데 일각에서는 정책금리 인하론까지 대두되면서 유로화가 크게 절하되고 있다면  서 유로화의 달러화에 대한 흐름이 중장기적으로 변화하는 것으로 볼 수 있는지   물었으며, 이에 대해 관련부서에서는 국제 투자은행들이 달러화 대비 유로화 전망  을 기존의 강세에서 약세로 바꾸고 있으며 장기적으로 다시 강세로 반전되지는   않을 것이라는 의견이 지배적이라고 설명하였음.@@@동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운  데 달러화 보유에 따른 환차익 기대가 형성될 경우 원화 강세에 대한 시장의 기 

In [5]:
a = script_ls[10].split('.')
for i in a:
    print(i +'\n\n')

NameError: name 'script_ls' is not defined

In [12]:
from ekonlpy.sentiment import MPKO
def determiner_s(sentence, hawki_ls, dovis_ls):
    #문장의 톤을 구하는 함수. (문장 하나가 이 함수의 첫번째 변수로 들어옴.)
    hawki_cnt = 0
    dovis_cnt = 0
    mpko = MPKO(kind=1)
    tokens = mpko.tokenize(sentence)
    #문장별로 토큰화
    #print(tokens)
    for token in tokens:
        if token in hawki_ls.values:
            hawki_cnt += 1
            #print(token+" hawk")
            #토큰이 매파에 해당하면 매파 개수 카운트 +1
        if token in dovis_ls.values:
            dovis_cnt += 1
            #print(token+" dovis")
            #토큰이 비둘기파에 해당하면 비둘기파 개수 카운트 +1
    if hawki_cnt > dovis_cnt:
        return 1
        #매파 토큰이 더 많은 경우.
    elif hawki_cnt < dovis_cnt:
        return 0
        #비둘기파 토큰이 더 많은 경우.
    else:
        return -1
        #매파, 비둘키파 토큰의 수가 각각 같은 경우.

def determiner_p(p, hawki_ls, dovis_ls):
    tmp = p.split('.')
    # 마침표(.)을 단위로 문장으로 나눔
    hawki_cnt = 0
    dovis_cnt = 0
    for sentence in tmp:
        #print(sentence)
        det = determiner_s(sentence, hawki_ls, dovis_ls)
        #나눠진 문장을 문장톤을 구하는 함수(본 함수의 위 함수)에 변수로 넣어 실행
        if det == 1:
            hawki_cnt +=1
            #매파 문장으로 판별 경우, hawk_cnt +1
        elif det == 0:
            dovis_cnt += 1
            #비둘기파 문장으로 판별 경우, dovis_cnt +1
        else:
            continue
    print('hawki : ' + str(hawki_cnt))
    print('dovis : ' + str(dovis_cnt))
    #print("\nsentence end")
    try:
        tone = (hawki_cnt - dovis_cnt)/(hawki_cnt + dovis_cnt)
        #문서 톤 구하기
        return tone
    except:
        print("tone error")
        return None

In [8]:
a = determiner_p('한국감정원 강여정 부장은 "정부의 11·3 대책 발표와 미국 기준금리 인상, 대출 규제 이후 주택시장이 관망세로 접어든 상태"라며 "계절적 비수기에 다 국내 산업경기 회복 지연, 공급과잉 우려 등까지 겹치면서 본격적인 봄이사철이 오기 전까지는 주택 가격이 상승하기 어려울 것"이라고 말했다', hawki_ls, dovis_ls)
print(a)
#임의 데이터로 테스트

산업/NNG;경기/NNG;회복/NNG hawk
금리/NNG;인상/NNG hawk
hawki : 1
dovis : 0

sentence end
1.0


In [13]:
script_ls = df['For+Fin']
tone_ls = []
for p in script_ls:
    p = p.replace('@@@','')
    tone = determiner_p(p,hawki_ls, dovis_ls)
    #determiner_p 함수로 문서 전체에 대한 톤 구함
    tone_ls.append(tone)
    #임시 리스트에 tone 저장
    print(tone)

hawki : 34
dovis : 24
0.1724137931034483
hawki : 36
dovis : 27
0.14285714285714285
hawki : 49
dovis : 22
0.38028169014084506
hawki : 36
dovis : 22
0.2413793103448276
hawki : 56
dovis : 20
0.47368421052631576
hawki : 44
dovis : 22
0.3333333333333333
hawki : 59
dovis : 13
0.6388888888888888
hawki : 54
dovis : 23
0.4025974025974026
hawki : 47
dovis : 15
0.5161290322580645
hawki : 61
dovis : 12
0.6712328767123288
hawki : 49
dovis : 12
0.6065573770491803
hawki : 51
dovis : 12
0.6190476190476191
hawki : 60
dovis : 19
0.5189873417721519
hawki : 58
dovis : 14
0.6111111111111112
hawki : 70
dovis : 12
0.7073170731707317
hawki : 60
dovis : 8
0.7647058823529411
hawki : 42
dovis : 19
0.3770491803278688
hawki : 47
dovis : 21
0.38235294117647056
hawki : 63
dovis : 16
0.5949367088607594
hawki : 55
dovis : 11
0.6666666666666666
hawki : 50
dovis : 14
0.5625
hawki : 54
dovis : 20
0.4594594594594595
hawki : 50
dovis : 17
0.4925373134328358
hawki : 53
dovis : 15
0.5588235294117647
hawki : 46
dovis : 17
0.4

In [14]:
tmp_ls #문서별 tone들이 저장된 임시 리스트

1      일부 위원은 우리나라 경제가 일본경제  에 비해 더 나아질 특별한 요인이 없음에도 ...
2      일부 위원은 6월 들어 국제유가가 크게   오르고 원화와 엔화간 동조화 현상이 뚜렷...
3      일부 위원은 위안화 절상 이후 아시아   통화중 우리나라 원화의 절상폭이 가장 컸다...
4      일부 위원은 최근 인도네시아 금융불안   사태가 발생한 가운데 국제 환투기세력이 인...
5      일부 위원은 미국과 우리나라의 정책금  리 격차가 확대되고 있는 가운데 금년 두 차...
6      일부 위원은 미 달러화에 대한 엔화 약  세가 지속될 것으로 보는지 물었으며, 이에...
7      일부 위원은 금년 들어 달러화가 엔화  나 유로화에 대해 상당히 강세를 나타낸 것과...
8      일부 위원은 2002년부터 거의 예외 없  이 연초에는 원화의 강세 현상이 나타나 ...
9      일부 위원은 자본유입에 대해서는 관대  한 반면 자본유출에 대해서는 비교적 엄격한 ...
10     일부 위원은 과거 사례를 보면 원화환  율은 엔화환율 및 위안화환율과 높은 상관관계...
11     일부 위원은 최근의 환율하락은 국내   수급사정의 변화 외에 시장참가자들의 정책금리...
12     일부 위원은 최근 은행들의 외화차입   확대가 환율하락 기대에 주로 기인한 것인지 ...
13     일부 위원은 세계적인 긴축기조의 영향  으로 신흥시장(emerging market)...
14     일부 위원은 지난 6월말 외평채 가산  금리가 80bp에 이른 것이 북한의 미사일 ...
15     일부 위원은 최근 외화대출이 대폭 증  가하였는데 은행들이 외화대출 재원 마련을 위...
16     일부 위원은 최근 원화선물옵션의 시  카고상업거래소(CME: Chicago Merc...
17     일부 위원은 최근 원/엔 환율의 하락은   금년내 일본의 금리 인상 가능성이 점차 ...
18     일부 위원은 최근 주요국 중앙은행들  이 외

In [15]:
tone_ls #문서별 tone들이 저장된 임시 리스트

[0.1724137931034483,
 0.14285714285714285,
 0.38028169014084506,
 0.2413793103448276,
 0.47368421052631576,
 0.3333333333333333,
 0.6388888888888888,
 0.4025974025974026,
 0.5161290322580645,
 0.6712328767123288,
 0.6065573770491803,
 0.6190476190476191,
 0.5189873417721519,
 0.6111111111111112,
 0.7073170731707317,
 0.7647058823529411,
 0.3770491803278688,
 0.38235294117647056,
 0.5949367088607594,
 0.6666666666666666,
 0.5625,
 0.4594594594594595,
 0.4925373134328358,
 0.5588235294117647,
 0.4603174603174603,
 0.6444444444444445,
 0.6631578947368421,
 0.4318181818181818,
 0.3253012048192771,
 0.5058823529411764,
 0.014492753623188406,
 0.2112676056338028,
 -0.02857142857142857,
 0.1044776119402985,
 0.15789473684210525,
 -0.13725490196078433,
 0.21875,
 0.4567901234567901,
 0.40540540540540543,
 0.09375,
 -0.5076923076923077,
 -0.6385542168674698,
 -0.6790123456790124,
 -0.76,
 -0.6842105263157895,
 -0.4915254237288136,
 -0.45454545454545453,
 -0.13636363636363635,
 0.035714285714285

In [16]:
df['tone'] = tone_ls #dataframe에 tone column 추가.

In [17]:
df

,date,For+Fin,tone
1,20050609,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,0.172414
2,20050707,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,0.142857
3,20050811,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,0.380282
4,20050908,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,0.241379
5,20051011,일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,0.473684
6,20051110,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",0.333333
7,20051208,일부 위원은 금년 들어 달러화가 엔화 나 유로화에 대해 상당히 강세를 나타낸 것과...,0.638889
8,20060112,일부 위원은 2002년부터 거의 예외 없 이 연초에는 원화의 강세 현상이 나타나 ...,0.402597
9,20060209,일부 위원은 자본유입에 대해서는 관대 한 반면 자본유출에 대해서는 비교적 엄격한 ...,0.516129
10,20060309,일부 위원은 과거 사례를 보면 원화환 율은 엔화환율 및 위안화환율과 높은 상관관계...,0.671233


In [19]:
df.to_csv("../DATA/tone-test-jh-190529.csv")

In [20]:
df2 = df[['date', 'tone']]
df2.to_csv("../DATA/tone-test-simple-190529.csv") #date와 tone만 저장된 파일입니다
df2

,date,tone
1,20050609,0.172414
2,20050707,0.142857
3,20050811,0.380282
4,20050908,0.241379
5,20051011,0.473684
6,20051110,0.333333
7,20051208,0.638889
8,20060112,0.402597
9,20060209,0.516129
10,20060309,0.671233


In [ ]:
df